# Задача

Зачет:

Обучить деттектор на 2 класса ```cat``` и  ```horse``` на базе датасета pascal VOC 2007, в качестве детектора предлагается взять  fasterrcnn_resnet50_fpn

Экзамен задача на выбор :

1. Применить к обученному детектору из зачета прунинг сверточных слоев L2 и дообучить модель после прунинга
2. Обучить модель по принципу RL игре Pong-v0

Зачет принимается индивидуально

Экзамен можно сдавать по группам не более 3-х человек

В итоге нужно прислать тетрадку с обучением и веса моделей

В данной работе продемонстрирую первую стратегию - Однократное прореживание

In [4]:
#training_dataset = torch.utils.data.TensorDataset(X_train, y_train)
#train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=50, shuffle=False)

### 1.2 Импортирование модулей

In [5]:
import numpy as np                       
import torch                       
import torchvision                       
from torch import nn                       
from torch.autograd import Variable                       
from torchvision.datasets import MNIST 
from torchvision.datasets import CIFAR10
import torchvision.transforms as T                      
from torchvision.utils import save_image
import matplotlib.pyplot as plt
from collections import defaultdict
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import time
from nni.algorithms.compression.pytorch.pruning import L1FilterPruner
from nni.algorithms.compression.pytorch.pruning import L2FilterPruner
from nni.compression.pytorch import ModelSpeedup

[2022-04-29 00:07:46] INFO (numexpr.utils/MainThread) NumExpr defaulting to 8 threads.


### 1.3 Воспроизводимое обучение

Для того, чтобы делать А-Б тесты и постепенно улучшать процесс обучения нейронной сети, необходимо позаботиться о том, чтобы результат обучения воспроизводился от запуска к запуску. Поскольку при обучении нейронных сетей часто используются псевдослучайные числа, необходимо, чтобы генераторы случайных чисел выдавали одни и те же последовательности от запуска к запуску. Кроме того, необходимо переключить CUDA в детерминированный режим. Это требование уменьшает скорость выполнения программы, зато результаты вычислений становятся воспроизводимыми.

In [6]:
seed=42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### 2.1 Определим функции для обучения

Класс ```MetricMonitor``` вспомогательный, который агригирует значения по каждому новому batch и выводит статистику онлайн накопительным образом. Это позволяет во время обучения мониторить изменение метрик онлайн.


Функция ```train``` отвечает за обучение. 

При этом, функция универсальная, тк работает с входными переменными: 

датасет ```train_loader```, 

модель ```model```, 

функцию ошибок ```criterion```, 

функцию оптимизации ```optimizer```, 

число эпох для обучения ```epochs```, 

params


Функция ```validate```  отчечает за прогон данных на валидационной выборке ```val_loader``` и так же универсальна, тк большую часть переменных получает извне.

In [7]:
class MetricMonitor:
    def __init__(self, float_precision=3):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0, "count": 0, "avg": 0})

    def update(self, metric_name, val):
        metric = self.metrics[metric_name]

        metric["val"] += val
        metric["count"] += 1
        metric["avg"] = metric["val"] / metric["count"]

    def __str__(self):
        return " | ".join(
            [
                "{metric_name}: {avg:.{float_precision}f}".format(
                    metric_name=metric_name, avg=metric["avg"], float_precision=self.float_precision
                )
                for (metric_name, metric) in self.metrics.items()
            ]
        )
    
def train(train_loader, model, criterion, optimizer, epochs, params):
    metric_monitor = MetricMonitor()
    model.train()
    eterator = tqdm(train_loader)
    for i, data in enumerate(eterator, start=1):
        
        image, label = data
        image = image.to(params["device"], non_blocking=True)
        label = label.to(params["device"], non_blocking=True)
        #print(type(image))
        output = model(image)
        pred = output.argmax(dim=1, keepdim=True)
        loss = criterion(output, label)
        metric_monitor.update("Loss", loss.item())
        metric_monitor.update("Accuracy", pred.eq(label.view_as(pred)).sum().item()/pred.shape[0])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        eterator.set_description(
            "Epoch: {epoch}. Train.      {metric_monitor}".format(epoch=epochs, metric_monitor=metric_monitor)
        )
        
def validate(val_loader, model, criterion, epoch, params):
    metric_monitor = MetricMonitor()
    model.eval()
    eterator = tqdm(val_loader)
    
    with torch.no_grad():
        for i, (image, label) in enumerate(eterator, start=1):
            image = image.to(params["device"], non_blocking=True)
            label = label.to(params["device"], non_blocking=True)
            output = model(image)
            pred = output.argmax(dim=1, keepdim=True)
            loss = criterion(output, label)
            metric_monitor.update("Loss", loss.item())
            metric_monitor.update("Accuracy", pred.eq(label.view_as(pred)).sum().item()/pred.shape[0])
            eterator.set_description(
                "Epoch: {epoch}. Validation. {metric_monitor}".format(epoch=epoch, metric_monitor=metric_monitor)
            )
    return metric_monitor

### 2.2 Определим функцию паплайна для обучения

Функция ```train_and_validate``` инициализирует train и valid датасет, задает функцию потерь и фунцию оптимизации, затем последовательно в Эпохе вызывает ранее определенную функцию обучения и функцию валидации.
Обращаю внимание, что этот паплайн возвращает в качестве результата обученную модель.

In [8]:
def train_and_validate(model, train_dataset, val_dataset, params):    
    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=params["batch_size"],
                              num_workers=params["num_workers"],
                              pin_memory=True,
                              shuffle=True)
    val_loader = DataLoader(dataset=val_dataset,
                              batch_size=params["batch_size"],
                              num_workers=params["num_workers"],
                              pin_memory=True,
                              shuffle=True)
    
    criterion =  nn.CrossEntropyLoss().to(params["device"])
    optimizer = torch.optim.AdamW(model.parameters(), lr=params["lr"])
    #print(params["epochs"] + 1)
    for epoch in range(1, params["epochs"] + 1):
        train(train_loader, model, criterion, optimizer, epoch, params)
        validate(val_loader, model, criterion, epoch, params)
    return model


### 3.1 Инициализация датасетов

В этом ноутбуке мы не будем писать своих датасетов, а будем использовать те, что имеются в наличии во вспомогательной библиотеке `torchvision`. Нас будет интересовать датасет `Cifar10` (10 классов).

1. Изучите интерфейс класса `torchvision.datasets.CIFAR10`. Посмотрите, какими параметрами можно влиять этот датасет.

2. Инициализируйте тренировочный и тестовый датасеты (`torchvision.datasets.CIFAR10`). 

3. В конструкторе используйте следующие параметры:
    * аугментации из `torchvision.transforms`:
        * `T.ToTensor()`
        * `T.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)),`

    * `root` -- любой на Ваш вкус
    * проанализируйте и выставьте остальные аргументы так, как вам кажется правильнее. Не забывайте, что перемешивать датасет имеет смысл для тренировочной выборки.


In [9]:
import torch
import torchvision

In [10]:
#px = 256
#MemoryError: Unable to allocate 1.50 MiB for an array with shape (3, 256, 256) and data type object
px = 32
preprocess = T.Compose([
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    T.Resize((px, px)),
    #T.Lambda(lambda x: torch.reshape(x,(32, 32, 3)))
    # torch.reshape меняет num_channels
])

In [11]:
#data_t = CIFAR10(root="./Cifar",train = True, transform = preprocess, download=True)
#data_v = CIFAR10(root="./Cifar",train = False, transform = preprocess, download=True)

In [12]:
data_train = torchvision.datasets.VOCDetection(root="VOC", year = '2007', image_set = 'train' , download=True, transform = preprocess)
data_test = torchvision.datasets.VOCDetection(root="VOC", year = '2007', image_set = 'val' , download=True, transform = preprocess)

Using downloaded and verified file: VOC\VOCtrainval_06-Nov-2007.tar
Extracting VOC\VOCtrainval_06-Nov-2007.tar to VOC
Using downloaded and verified file: VOC\VOCtrainval_06-Nov-2007.tar
Extracting VOC\VOCtrainval_06-Nov-2007.tar to VOC


In [13]:
torchvision.transforms.functional.get_image_num_channels(data_train[0][0])

3

In [14]:
#data_t

In [15]:
#data_train.__dict__

In [16]:
#data_t.__dict__

In [17]:
#plt.imshow(data_t.data[6])

In [18]:
#np.shape(data_t.data[6])

In [19]:
#np.shape(data_train[0][0])

In [20]:
#plt.imshow(data_train[0][0])

In [21]:
#data_train_reshape = [(torch.reshape(data_train[i][0], (32, 32, 3)), i) for i in range(len(data_train))]

In [22]:
#data_test_reshape = [(torch.reshape(data_test[i][0], (32, 32, 3)), i) for i in range(len(data_train))]

In [23]:
#type(data_t[3]), type(data_t)

In [24]:
#type(data_train_reshape[3]), type(data_train_reshape)

In [25]:
#data_train_reshape.targets

In [26]:
#data_train[0][0]

In [27]:
#plt.imshow(data_train.images[0])

In [28]:
#np.shape(data_train.data[6])

In [29]:
#data_train[0][1]

In [30]:
#type(data_train[0][0])

In [31]:
#np.shape(data_train[0][0])

In [32]:
#np.shape(data_train[1][0])

In [33]:
#np.shape(data_train[3][0])

In [34]:
#data_train[17][1]['annotation']['object'][0]['name']

In [35]:
#dir(data_t[1])
#data_t[1].count, data_t[1].index

#data_t[1][1]

In [36]:
#data_train_conv = np.empty([len(data_train), 2])
arr_train = []
for i in range(len(data_train)):
    #data = data_train[i][0]
    if data_train[i][1]['annotation']['object'][0]['name'] == 'cat':
        label = 0
    elif data_train[i][1]['annotation']['object'][0]['name'] == 'horse':
        label = 1
    else:
        label = 3
    #ValueError: only one element tensors can be converted to Python scalars
    #data_train_conv[i][0] = data_train[i][0]
    #data_train_conv[i][1] = label
    tup = (data_train[i][0], label)
    arr_train.append(tup)
data_train_conv=np.array(arr_train)

C:\Users\anike\AppData\Local\Temp/ipykernel_11272/3520638920.py:16: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  data_train_conv=np.array(arr_train)
C:\Users\anike\AppData\Local\Temp/ipykernel_11272/3520638920.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data_train_conv=np.array(arr_train)


In [37]:
#data_train_conv=np.array(arr)

In [38]:
#arr[0]

In [39]:
#data_train_conv

In [40]:
arr_test=[]
for i in range(len(data_test)):
    #data = data_train[i][0]
    if data_test[i][1]['annotation']['object'][0]['name'] == 'cat':
        label = 0
    elif data_test[i][1]['annotation']['object'][0]['name'] == 'horse':
        label = 1
    else:
        label = 3
    tup = (data_test[i][0], label)
    arr_test.append(tup)
data_test_conv=np.array(arr_test, dtype=object)

C:\Users\anike\AppData\Local\Temp/ipykernel_11272/433263208.py:12: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  data_test_conv=np.array(arr_test, dtype=object)


In [41]:
type(data_train_conv[0][0]), type(data_train_conv[0][1])

(torch.Tensor, int)

**По итогу получаем 3 класса:**

0. cat
1. horse
2. not(cat or horse)

### 4.1 Проведем файн-тюнинг модели

Для скорости обучения возьмем стандартную предобученную легкую модель ResNet18 из библиотеки ```torchvision```. Нам потребуется изменить последний линейный слой ```fc``` на слой с 10 ответами, аналогично ответам в датасете.

In [42]:
model = torchvision.models.resnet18(pretrained= True)

In [43]:
params = {
    "device": "cpu",
    "lr": 0.0001,
    "batch_size": 128,
    "num_workers": 8,
    "epochs": 5,
}

In [44]:
model = torchvision.models.resnet18(pretrained= True)
model.fc = nn.Linear(in_features=512, out_features=10, bias=True)
model = model.to(params["device"])

Зададим прочие параметры обучения в объект типа dict ```params```

И запустим обучение

In [45]:
#model = train_and_validate(model, data_train, data_test, params)
#TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object

In [46]:
#dtr = T.Compose(data_train)
#dts = T.Compose(data_test)
#model = train_and_validate(model, dtr, dts, params)
#TypeError: object of type 'Compose' has no len()

In [47]:
#type(data_t), type(data_train_conv), type(arr)

In [48]:
#print(type(data_t[0]),type(data_t[1]))
#print(type(data_train_conv[0]),type(data_train_conv[1]))
#print(type(arr[0]),type(arr[1]))

In [49]:
#data_train_conv[0]

In [50]:
#print(type(type(data_t[0][0]),type(data_t[1][1])))
#print(type(type(data_train_conv[0][0]),type(data_train_conv[1][1])))

In [51]:
#PicklingError: Can't pickle <function <lambda> at 0x00000247817AF0D0>: attribute lookup <lambda> on __main__ failed
#model = train_and_validate(model, data_train, data_test, params)

In [52]:
# RuntimeError: Given groups=1, 
# weight of size [64, 3, 7, 7], 
# expected input[128, 32, 32, 3] 
# to have 3 channels, but got 32 channels instead
#model = train_and_validate(model, arr_train, arr_test, params)

In [53]:
model = train_and_validate(model, arr_train, arr_test, params)

Epoch: 1. Train.      Loss: 1.959 | Accuracy: 0.425: 100%|█████████████████████████████| 20/20 [00:44<00:00,  2.24s/it]
Epoch: 1. Validation. Loss: 1.816 | Accuracy: 0.639: 100%|█████████████████████████████| 20/20 [00:13<00:00,  1.53it/s]
Epoch: 2. Train.      Loss: 0.791 | Accuracy: 0.901: 100%|█████████████████████████████| 20/20 [00:45<00:00,  2.26s/it]
Epoch: 2. Validation. Loss: 0.763 | Accuracy: 0.870: 100%|█████████████████████████████| 20/20 [00:12<00:00,  1.55it/s]
Epoch: 3. Train.      Loss: 0.329 | Accuracy: 0.954: 100%|█████████████████████████████| 20/20 [00:44<00:00,  2.22s/it]
Epoch: 3. Validation. Loss: 0.547 | Accuracy: 0.881: 100%|█████████████████████████████| 20/20 [00:12<00:00,  1.57it/s]
Epoch: 4. Train.      Loss: 0.166 | Accuracy: 0.978: 100%|█████████████████████████████| 20/20 [00:43<00:00,  2.16s/it]
Epoch: 4. Validation. Loss: 0.516 | Accuracy: 0.881: 100%|█████████████████████████████| 20/20 [00:12<00:00,  1.59it/s]
Epoch: 5. Train.      Loss: 0.091 | Accu

In [54]:
#print(type(data_t[0]),type(data_t[1]))
#print(type(arr[0]),type(arr[1]))

In [55]:
#print(type(data_t[0][0]),type(data_t[0][1]))
#print(type(arr[0][0]),type(arr[0][1]))

In [56]:
#np.shape(data_t[0][0])

In [57]:
#np.shape(arr[0][0])

In [58]:
#(torchvision.transforms.functional.get_image_num_channels(data_t[0][0]),
# torchvision.transforms.functional.get_image_num_channels(arr[0][0]))

In [59]:
#Can't pickle <function <lambda> at 0x00000247817AF0D0>: attribute lookup <lambda> on __main__ failed
#model = train_and_validate(model, data_t, data_v, params)

In [60]:
#The input are organized in [N, C, W, H]
#model = train_and_validate(model, arr_train, arr_test, params)

In [61]:
#model = torchvision.models.detection.fasterrcnn_resnet50_fpn(num_classes=2)

In [62]:
#model = torchvision.models.resnet18(pretrained= True)
#model.fc = nn.Linear(in_features=512, out_features=10, bias=True)
#model = model.to(params["device"])

In [63]:
# ValueError: In training mode, targets should be passed
# output = model(image)
# model = train_and_validate(model, arr_train, arr_test, params)

In [64]:
torch.save(model, "./model.pth")

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object

### 5.1 Оптимизация(прунинг) ранее обученно модели

Проведем оптимизацию моделей двумя разными методами для сравнения ```L1FilterPruner``` и ```L2FilterPruner```, сохраним результаты пруненых моделей "model_prunel1.pth" и "model_prunel2.pth"

**По задаче только L2**

In [65]:
#config_list = [{ 'sparsity': 0.5, 
#                'op_types': ['Conv2d'] 
#               }]
#model = torch.load ("./model.pth", map_location="cpu").eval()
#dummy_input = torch.randn(1, 3, px, px).to("cpu")
#pruner = L1FilterPruner(model, config_list); #, dependency_aware=True, dummy_input=dummy_input);
#pruned_model = pruner.compress();
#pruner.export_model(model_path='./test.pt', mask_path='./test_mask.pt')
#m = torch.load ("./model.pth", map_location="cpu").eval()
#m_speedup = ModelSpeedup(m, dummy_input, './test_mask.pt')
#m_speedup.speedup_model();
#torch.save(m, "./model_prunel1.pth")

In [66]:
config_list = [{ 'sparsity': 0.5, 'op_types': ['Conv2d'] }]
model = torch.load ("./model.pth", map_location="cpu").eval()
dummy_input = torch.randn(1, 3, px, px).to("cpu")
pruner = L2FilterPruner(model, config_list) #, dependency_aware=True, dummy_input=dummy_input);
pruned_model = pruner.compress();
pruner.export_model(model_path='./test.pt', mask_path='./test_mask.pt')
m = torch.load ("./model.pth", map_location="cpu").eval()
m_speedup = ModelSpeedup(m, dummy_input, './test_mask.pt')
m_speedup.speedup_model();
torch.save(m, "./model_prunel2.pth")

[2022-04-29 00:15:55] INFO (nni.compression.pytorch.compressor/MainThread) Model state_dict saved to ./test.pt
[2022-04-29 00:15:55] INFO (nni.compression.pytorch.compressor/MainThread) Mask dict saved to ./test_mask.pt
[2022-04-29 00:15:56] INFO (nni.compression.pytorch.speedup.compressor/MainThread) start to speedup the model
[2022-04-29 00:15:56] INFO (FixMaskConflict/MainThread) {'conv1': 1, 'layer1.0.conv1': 1, 'layer1.0.conv2': 1, 'layer1.1.conv1': 1, 'layer1.1.conv2': 1, 'layer2.0.conv1': 1, 'layer2.0.conv2': 1, 'layer2.0.downsample.0': 1, 'layer2.1.conv1': 1, 'layer2.1.conv2': 1, 'layer3.0.conv1': 1, 'layer3.0.conv2': 1, 'layer3.0.downsample.0': 1, 'layer3.1.conv1': 1, 'layer3.1.conv2': 1, 'layer4.0.conv1': 1, 'layer4.0.conv2': 1, 'layer4.0.downsample.0': 1, 'layer4.1.conv1': 1, 'layer4.1.conv2': 1}
[2022-04-29 00:15:57] INFO (FixMaskConflict/MainThread) dim0 sparsity: 0.500000
[2022-04-29 00:15:57] INFO (FixMaskConflict/MainThread) dim1 sparsity: 0.000000
[2022-04-29 00:15:57]

[2022-04-29 00:15:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update mask for layer4.1.conv2
[2022-04-29 00:15:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update mask for layer4.1.bn2
[2022-04-29 00:15:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update mask for layer4.1.aten::add_.68
[2022-04-29 00:15:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update mask for layer4.1.relu
[2022-04-29 00:15:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update mask for avgpool
[2022-04-29 00:15:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update mask for .aten::flatten.60
[2022-04-29 00:15:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update mask for fc
[2022-04-29 00:15:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the fc


C:\Users\anike\anaconda3\lib\site-packages\torch\_tensor.py:1104: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:475.)
  return self._grad


[2022-04-29 00:15:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the .aten::flatten.60
[2022-04-29 00:15:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the avgpool
[2022-04-29 00:15:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the layer4.1.relu.1
[2022-04-29 00:15:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the layer4.1.aten::add_.68
[2022-04-29 00:15:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the layer4.1.bn2
[2022-04-29 00:15:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the layer4.1.conv2
[2022-04-29 00:15:57] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the layer4.1.relu
[2022-04-29 00:15:57] INFO (nni.compression.pytorch.s

[2022-04-29 00:15:58] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the layer1.0.relu
[2022-04-29 00:15:58] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the layer1.0.bn1
[2022-04-29 00:15:58] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the layer1.0.conv1
[2022-04-29 00:15:58] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the maxpool
[2022-04-29 00:15:58] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the relu
[2022-04-29 00:15:58] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the bn1
[2022-04-29 00:15:58] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Update the indirect sparsity for the conv1
[2022-04-29 00:15:58] INFO (nni.compression.pytorch.speedup.compressor/MainThread) resolve the 

[2022-04-29 00:15:58] INFO (nni.compression.pytorch.speedup.compressor/MainThread) replace module (name: layer3.0.conv2, op_type: Conv2d)
[2022-04-29 00:15:58] INFO (nni.compression.pytorch.speedup.compressor/MainThread) replace module (name: layer3.0.bn2, op_type: BatchNorm2d)
[2022-04-29 00:15:58] INFO (nni.compression.pytorch.speedup.compress_modules/MainThread) replace batchnorm2d with num_features: 238
[2022-04-29 00:15:58] INFO (nni.compression.pytorch.speedup.compressor/MainThread) Warning: cannot replace (name: layer3.0.aten::add_.65, op_type: aten::add_) which is func type
[2022-04-29 00:15:58] INFO (nni.compression.pytorch.speedup.compressor/MainThread) replace module (name: layer3.0.relu, op_type: ReLU)
[2022-04-29 00:15:58] INFO (nni.compression.pytorch.speedup.compressor/MainThread) replace module (name: layer3.1.conv1, op_type: Conv2d)
[2022-04-29 00:15:58] INFO (nni.compression.pytorch.speedup.compressor/MainThread) replace module (name: layer3.1.bn1, op_type: BatchNorm2

### 5.2 Анализ результатов

Проведем анализ получившихся результатов по рамеру моделей и  скорости работы. Для этого создадим функцию оценки количества весов модели
```get_n_params```

In [67]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [68]:
#model_prunel1 = torch.load ("./model_prunel1.pth", map_location="cpu").eval()
model_prunel2 = torch.load ("./model_prunel2.pth", map_location="cpu").eval()
model = torch.load ("./model.pth", map_location="cpu").eval()
#dummy_input = torch.ones(128, 3, 32, 32).to("cpu")  [64, 3, 7, 7]
dummy_input = torch.ones(64, 3, 7, 7).to("cpu") 

In [69]:
get_n_params(model)

11181642

In [70]:
#get_n_params(model_prunel1)

In [71]:
get_n_params(model_prunel2)

5184546

In [72]:
#print ("L1pruning {metric1}%, L2pruning {metric2}% parametr of original".format( \
#                                    metric1=round(get_n_params(model_prunel1)/get_n_params(model)*100,2), \
#                                    metric2=round(get_n_params(model_prunel2)/get_n_params(model)*100,2)))
print ("L2pruning {metric2}% parametr of original".format(metric2=round(get_n_params(model_prunel2)/get_n_params(model)*100,2)))

L2pruning 46.37% parametr of original


In [73]:
%%timeit
model(dummy_input)

39.2 ms ± 1.52 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [74]:
#%%timeit
#model_prunel1(dummy_input)

In [75]:
%%timeit
model_prunel2(dummy_input)

21.3 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 5.3 Дообучение пруниных моделей

После прунинга точность моделей заметно уменьшается, поэтому требуется дообучение. 

In [76]:
params = {
    "device": "cpu",
    "lr": 0.0001,
    "batch_size": 128,
    "num_workers": 8,
    "epochs": 15,
}

In [77]:
#model_prunel1 = train_and_validate(model_prunel1.to("cpu"), arr_train, arr_test, params)

In [79]:
#model_prunel2 = train_and_validate(model_prunel2.to("cuda"), arr_train, arr_test, params)
#AssertionError: Torch not compiled with CUDA enabled

# conda install -c pytorch pytorch=1.2.0=py3.7_cuda92_cudnn7_1 
# conda install pytorch torchvision cudatoolkit=10.1 -c pytorch
# устанавливала, не помогло

In [80]:
model_prunel2 = train_and_validate(model_prunel2.to("cpu"), arr_train, arr_test, params)

Epoch: 1. Train.      Loss: 0.516 | Accuracy: 0.895: 100%|█████████████████████████████| 20/20 [00:25<00:00,  1.27s/it]
Epoch: 1. Validation. Loss: 0.510 | Accuracy: 0.889: 100%|█████████████████████████████| 20/20 [00:11<00:00,  1.72it/s]
Epoch: 2. Train.      Loss: 0.272 | Accuracy: 0.925: 100%|█████████████████████████████| 20/20 [00:24<00:00,  1.25s/it]
Epoch: 2. Validation. Loss: 0.482 | Accuracy: 0.876: 100%|█████████████████████████████| 20/20 [00:11<00:00,  1.75it/s]
Epoch: 3. Train.      Loss: 0.166 | Accuracy: 0.953: 100%|█████████████████████████████| 20/20 [00:24<00:00,  1.23s/it]
Epoch: 3. Validation. Loss: 0.478 | Accuracy: 0.873: 100%|█████████████████████████████| 20/20 [00:11<00:00,  1.75it/s]
Epoch: 4. Train.      Loss: 0.104 | Accuracy: 0.973: 100%|█████████████████████████████| 20/20 [00:24<00:00,  1.25s/it]
Epoch: 4. Validation. Loss: 0.497 | Accuracy: 0.869: 100%|█████████████████████████████| 20/20 [00:11<00:00,  1.69it/s]
Epoch: 5. Train.      Loss: 0.061 | Accu

Число весов

In [81]:
get_n_params(model_prunel2)

5184546

Время

In [82]:
%%timeit
model_prunel2(dummy_input)

20.1 ms ± 850 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [87]:
#model_prunel2.__dict__

In [84]:
model_prunel2.parameters()

<generator object Module.parameters at 0x0000026BED889660>

In [88]:
len(list(model_prunel2.parameters()))

62

Веса [0]

In [86]:
list(model_prunel2.parameters())[0]

Parameter containing:
tensor([[[[-9.7890e-03, -5.0687e-03, -1.2570e-03,  ...,  5.5386e-02,
            1.5564e-02, -1.2600e-02],
          [ 1.0369e-02,  9.8975e-03, -1.1004e-01,  ..., -2.7271e-01,
           -1.3055e-01,  3.2204e-03],
          [-7.2177e-03,  5.8775e-02,  2.9488e-01,  ...,  5.1752e-01,
            2.5316e-01,  6.2023e-02],
          ...,
          [-2.6926e-02,  1.7066e-02,  7.3154e-02,  ..., -3.3308e-01,
           -4.2170e-01, -2.5870e-01],
          [ 3.0488e-02,  4.0612e-02,  6.2570e-02,  ...,  4.1319e-01,
            3.9261e-01,  1.6574e-01],
          [-1.3941e-02, -3.2089e-03, -2.4613e-02,  ..., -1.5259e-01,
           -8.3408e-02, -7.9415e-03]],

         [[-1.0284e-02, -2.5085e-02, -3.3558e-02,  ...,  3.1665e-02,
           -4.7774e-05, -2.5233e-02],
          [ 4.5777e-02,  3.4507e-02, -1.0396e-01,  ..., -3.1355e-01,
           -1.6088e-01, -1.8309e-03],
          [-2.6971e-04,  9.9565e-02,  4.0197e-01,  ...,  7.0684e-01,
            3.6674e-01,  1.2343e-01]